# Audit System with Simplified JSON Input/Output

This notebook implements an audit system that:
1. Takes input data in JSON format
2. Processes disruptions and reassignments
3. Returns output in both human-readable and JSON formats

## Data Structure

### Input JSON Format
```json
{
    "auditors": [
        {
            "auditor_id": "A001",
            "latitude": 40.7128,
            "longitude": -74.0060,
            "availability_status": "Available"
        }
    ],
    "stores": [
        {
            "store_id": "S001",
            "latitude": 40.7120,
            "longitude": -74.0052,
            "store_status": "Open",
            "assigned_auditor_id": "A001"
        }
    ],
    "disruptions": []
}
```

In [2]:
print("hello world!")

hello world!


In [10]:
import json
from math import sqrt
from datetime import datetime

# ---------- DUMMY DATA ----------
dummy_data = {
    "auditors": [
        {
            "auditor_id": "A001",
            "latitude": 40.7128,
            "longitude": -74.0060,
            "availability_status": "Available"
        },
        {
            "auditor_id": "A002",
            "latitude": 40.7589,
            "longitude": -73.9851,
            "availability_status": "Unavailable"
        },
        {
            "auditor_id": "A003",
            "latitude": 40.7549,
            "longitude": -73.9840,
            "availability_status": "Available"
        }
    ],
    "stores": [
        {
            "store_id": "S001",
            "latitude": 40.7120,
            "longitude": -74.0052,
            "store_status": "Open",
            "assigned_auditor_id": "A001"
        },
        {
            "store_id": "S002",
            "latitude": 40.7580,
            "longitude": -73.9855,
            "store_status": "Closed",
            "assigned_auditor_id": "A002"
        },
        {
            "store_id": "S003",
            "latitude": 40.7540,
            "longitude": -73.9845,
            "store_status": "Open",
            "assigned_auditor_id": "A002"
        }
    ],
    "disruptions": []
}

# ---------- HELPER FUNCTIONS ----------

def calculate_distance(lat1, lon1, lat2, lon2):
    """Euclidean distance (approximation for short distances)"""
    return sqrt((lat1 - lat2) ** 2 + (lon1 - lon2) ** 2)

def find_best_auditor(store, auditors):
    """Find suitable auditor based on distance"""
    available_auditors = [a for a in auditors if a['availability_status'] == 'Available']
    
    if not available_auditors:
        return None

    scores = []
    for aud in available_auditors:
        distance = calculate_distance(
            store['latitude'], store['longitude'],
            aud['latitude'], aud['longitude']
        )
        scores.append((aud['auditor_id'], distance))

    best_auditor_id = sorted(scores, key=lambda x: x[1])[0][0]
    return best_auditor_id

def get_store_by_id(store_id, stores):
    """Get store by ID"""
    return next((s for s in stores if s['store_id'] == store_id), None)

def get_auditor_by_id(auditor_id, auditors):
    """Get auditor by ID"""
    return next((a for a in auditors if a['auditor_id'] == auditor_id), None)

def reassign_store(store_id, old_auditor_id, auditors, stores):
    """Reassign one store"""
    store = get_store_by_id(store_id, stores)
    if not store:
        print(f"Store {store_id} not found")
        return None

    best_auditor_id = find_best_auditor(store, auditors)
    if best_auditor_id is None:
        print(f"No available auditor found for store {store_id}")
        return None

    print(f"\n🔁 Reassigning Store {store_id}:")
    print(f"   Before → assigned_auditor_id: {old_auditor_id}")
    print(f"   Candidate auditor: {best_auditor_id}")

    # Update assignment
    store['assigned_auditor_id'] = best_auditor_id

    print(f"   After → assigned_auditor_id: {best_auditor_id}")
    print(f"✅ Store {store_id} reassigned successfully!\n")

    return best_auditor_id

def handle_disruption(event_type, store_id, reported_by, auditors, stores):
    """Apply the detected disruption"""
    if event_type == "Auditor Unavailable":
        print(f"\n⚠️ Auditor {reported_by} is unavailable. Reassigning their stores...")
        affected_stores = [s for s in stores if s['assigned_auditor_id'] == reported_by]
        if not affected_stores:
            print(f"   No stores assigned to {reported_by}.")
        for store in affected_stores:
            reassign_store(store['store_id'], reported_by, auditors, stores)

    elif event_type == "Store Closed":
        store = get_store_by_id(store_id, stores)
        if store:
            before_status = store['store_status']
            print(f"\n⚠️ Store {store_id} is being closed.")
            print(f"   Before → store_status: {before_status}")
            store['store_status'] = 'Closed'
            print(f"   After  → store_status: {store['store_status']}")
            print(f"❌ Store {store_id} marked as Closed.\n")

def detect_disruptions(auditors, stores):
    """Detect disruptions based on current data directly"""
    disruptions = []

    # Detect unavailable auditors
    unavailable_auditors = [a for a in auditors if a['availability_status'] == 'Unavailable']
    for a in unavailable_auditors:
        disruptions.append({
            "event_type": "Auditor Unavailable",
            "store_id": None,
            "reported_by": a['auditor_id']
        })

    # Detect closed stores
    closed_stores = [s for s in stores if s['store_status'] == 'Closed']
    for s in closed_stores:
        disruptions.append({
            "event_type": "Store Closed",
            "store_id": s['store_id'],
            "reported_by": s['assigned_auditor_id']
        })

    return disruptions

def main(data):
    """Main function that processes the data"""
    print("🚀 Running Real-Time Auto-Detection + Reassignment System...\n")

    auditors = data['auditors']
    stores = data['stores']
    disruptions = data['disruptions']

    # Detect disruptions based on current data
    new_disruptions = detect_disruptions(auditors, stores)

    # Filter out ones already logged
    existing_ids = {d['reported_by'] + d['event_type'] for d in disruptions}
    new_events = [
        d for d in new_disruptions
        if d['reported_by'] + d['event_type'] not in existing_ids
    ]

    if not new_events:
        print("✅ No new disruptions detected. Everything is stable.\n")
    else:
        print(f"⚠️ {len(new_events)} new disruptions detected.\n")

    # Process new disruptions
    for d in new_events:
        disruption_id = f"D{len(disruptions) + 1:03d}"
        new_disruption = {
            "disruption_id": disruption_id,
            "store_id": d["store_id"],
            "event_type": d["event_type"],
            "triggered_on": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "reported_by": d["reported_by"],
            "reassignment_status": "Pending"
        }
        disruptions.append(new_disruption)

        handle_disruption(d['event_type'], d['store_id'], d['reported_by'], auditors, stores)
        new_disruption['reassignment_status'] = 'Implemented'

    # Create simplified output structure
    output_data = {
        "auditors": [{
            "auditor_id": a["auditor_id"],
            "latitude": a["latitude"],
            "longitude": a["longitude"],
            "availability_status": a["availability_status"]
        } for a in auditors],
        "stores": [{
            "store_id": s["store_id"],
            "latitude": s["latitude"],
            "longitude": s["longitude"],
            "store_status": s["store_status"],
            "assigned_auditor_id": s["assigned_auditor_id"]
        } for s in stores],
        "disruptions": disruptions
    }

    print("✅ Current system state:")
    print("\n--- Auditors ---")
    for a in output_data['auditors']:
        print(f"ID: {a['auditor_id']}, Status: {a['availability_status']}")
    print("\n--- Stores ---")
    for s in output_data['stores']:
        print(f"ID: {s['store_id']}, Status: {s['store_status']}, Auditor: {s['assigned_auditor_id']}")
    print("\n--- Recent Disruptions ---")
    for d in output_data['disruptions'][-5:]:
        print(f"ID: {d['disruption_id']}, Type: {d['event_type']}, Status: {d['reassignment_status']}")

    return output_data

In [11]:
# Example usage and JSON output
import json

def format_json_output(data):
    """Format and return the data as a JSON string"""
    return json.dumps(data, indent=2)

# Run the system with dummy data
result = main(dummy_data)

print("\n=== JSON Output ===")
print(format_json_output(result))

# Optionally save to file
with open('audit_system_state.json', 'w') as f:
    json.dump(result, f, indent=2)

🚀 Running Real-Time Auto-Detection + Reassignment System...

⚠️ 2 new disruptions detected.


⚠️ Auditor A002 is unavailable. Reassigning their stores...

🔁 Reassigning Store S002:
   Before → assigned_auditor_id: A002
   Candidate auditor: A003
   After → assigned_auditor_id: A003
✅ Store S002 reassigned successfully!


🔁 Reassigning Store S003:
   Before → assigned_auditor_id: A002
   Candidate auditor: A003
   After → assigned_auditor_id: A003
✅ Store S003 reassigned successfully!


⚠️ Store S002 is being closed.
   Before → store_status: Closed
   After  → store_status: Closed
❌ Store S002 marked as Closed.

✅ Current system state:

--- Auditors ---
ID: A001, Status: Available
ID: A002, Status: Unavailable
ID: A003, Status: Available

--- Stores ---
ID: S001, Status: Open, Auditor: A001
ID: S002, Status: Closed, Auditor: A003
ID: S003, Status: Open, Auditor: A003

--- Recent Disruptions ---
ID: D001, Type: Auditor Unavailable, Status: Implemented
ID: D002, Type: Store Closed, Statu

# Spring Boot Integration

This section implements a REST API endpoint using Flask to:
1. Accept JSON input from Spring Boot
2. Process the auditor assignments
3. Return JSON output back to Spring Boot

## API Endpoint
- URL: `http://localhost:5000/api/process-assignments`
- Method: POST
- Input: JSON with auditors and stores data
- Output: JSON with updated assignments and disruptions

In [ ]:
# Install required packages if not already installed
try:
    from flask import Flask, request, jsonify
    print("Flask is already installed")
except ImportError:
    # Use subprocess to install so this works from script and avoids notebook magics
    import sys, subprocess
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'flask'])
    from flask import Flask, request, jsonify

# Create Flask application
app = Flask(__name__)

class ValidationError(Exception):
    """Custom exception for validation errors"""
    pass

def validate_auditor(auditor):
    """Validate auditor data"""
    required_fields = ['auditor_id', 'latitude', 'longitude', 'availability_status']
    missing_fields = [field for field in required_fields if field not in auditor]
    if missing_fields:
        raise ValidationError(f"Auditor missing required fields: {', '.join(missing_fields)}")
    
    # Validate data types
    if not isinstance(auditor['auditor_id'], str):
        raise ValidationError("auditor_id must be a string")
    if not isinstance(auditor['latitude'], (int, float)):
        raise ValidationError("latitude must be a number")
    if not isinstance(auditor['longitude'], (int, float)):
        raise ValidationError("longitude must be a number")
    if not isinstance(auditor['availability_status'], str):
        raise ValidationError("availability_status must be a string")
    
    # Validate value ranges
    if not -90 <= auditor['latitude'] <= 90:
        raise ValidationError("latitude must be between -90 and 90")
    if not -180 <= auditor['longitude'] <= 180:
        raise ValidationError("longitude must be between -180 and 180")
    if auditor['availability_status'] not in ['Available', 'Unavailable']:
        raise ValidationError("availability_status must be 'Available' or 'Unavailable'")

def validate_store(store):
    """Validate store data"""
    required_fields = ['store_id', 'latitude', 'longitude', 'store_status']
    missing_fields = [field for field in required_fields if field not in store]
    if missing_fields:
        raise ValidationError(f"Store missing required fields: {', '.join(missing_fields)}")
    
    # Validate data types
    if not isinstance(store['store_id'], str):
        raise ValidationError("store_id must be a string")
    if not isinstance(store['latitude'], (int, float)):
        raise ValidationError("latitude must be a number")
    if not isinstance(store['longitude'], (int, float)):
        raise ValidationError("longitude must be a number")
    if not isinstance(store['store_status'], str):
        raise ValidationError("store_status must be a string")
    
    # Validate value ranges
    if not -90 <= store['latitude'] <= 90:
        raise ValidationError("latitude must be between -90 and 90")
    if not -180 <= store['longitude'] <= 180:
        raise ValidationError("longitude must be between -180 and 180")
    if store['store_status'] not in ['Open', 'Closed']:
        raise ValidationError("store_status must be 'Open' or 'Closed'")

@app.route('/api/process-assignments', methods=['POST'])
def process_assignments():
    try:
        # Get JSON data from Spring Boot
        data = request.get_json()
        if not data:
            return jsonify({
                'error': 'No JSON data provided',
                'status': 'error',
                'code': 'INVALID_REQUEST'
            }), 400
            
        # Validate basic structure
        if 'auditors' not in data or 'stores' not in data:
            return jsonify({
                'error': 'Missing required sections',
                'status': 'error',
                'code': 'MISSING_DATA',
                'required_fields': ['auditors', 'stores']
            }), 400
            
        # Validate each auditor and store
        try:
            for auditor in data['auditors']:
                validate_auditor(auditor)
            for store in data['stores']:
                validate_store(store)
        except ValidationError as ve:
            return jsonify({
                'error': str(ve),
                'status': 'error',
                'code': 'VALIDATION_ERROR'
            }), 400
            
        # Initialize disruptions list if not present
        if 'disruptions' not in data:
            data['disruptions'] = []
            
        # Process the assignments using our existing logic
        try:
            result = main(data)
        except Exception as e:
            return jsonify({
                'error': 'Error processing assignments',
                'status': 'error',
                'code': 'PROCESSING_ERROR',
                'details': str(e)
            }), 500
        
        # Validate output before sending
        if not result or not all(k in result for k in ['auditors', 'stores', 'disruptions']):
            return jsonify({
                'error': 'Invalid processing result',
                'status': 'error',
                'code': 'INVALID_RESULT'
            }), 500
        
        # Return success response
        return jsonify({
            'status': 'success',
            'code': 'SUCCESS',
            'data': result
        })
        
    except Exception as e:
        return jsonify({
            'error': 'Internal server error',
            'status': 'error',
            'code': 'INTERNAL_ERROR',
            'message': str(e)
        }), 500

# Start the Flask server if running directly
if __name__ == '__main__':
    print("Starting Flask server...")
    print("API endpoint will be available at: http://localhost:5000/api/process-assignments")
    app.run(host='0.0.0.0', port=5000)

Flask is already installed
Starting Flask server...
API endpoint will be available at: http://localhost:5000/api/process-assignments
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.1.46:5000/ (Press CTRL+C to quit)
 * Running on http://192.168.1.46:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [08/Nov/2025 11:12:51] "GET /api/process-assignments HTTP/1.1" 405 -
127.0.0.1 - - [08/Nov/2025 11:12:51] "GET /api/process-assignments HTTP/1.1" 405 -
127.0.0.1 - - [08/Nov/2025 11:12:51] "GET /api/process-assignments HTTP/1.1" 405 -
127.0.0.1 - - [08/Nov/2025 11:12:51] "GET /api/process-assignments HTTP/1.1" 405 -
127.0.0.1 - - [08/Nov/2025 11:12:51] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [08/Nov/2025 11:12:51] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [08/Nov/2025 11:13:06] "GET /api/process-assignments HTTP/1.1" 405 -
127.0.0.1 - - [08/Nov/2025 11:13:06] "GET /api/process-assignments HTTP/1.1" 405 -
127.0.0.1 - - [08/Nov/2025 11:13:06] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [08/Nov/2025 11:13:06] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [08/Nov/2025 11:13:25] "GET /api/process-assignme

# Usage with Spring Boot

## Spring Boot Controller Example
```java
@RestController
@RequestMapping("/api")
public class AuditorAssignmentController {
    
    @PostMapping("/assign-auditors")
    public ResponseEntity<?> assignAuditors(@RequestBody AssignmentRequest request) {
        try {
            // Call Python API
            String pythonApiUrl = "http://localhost:5000/api/process-assignments";
            
            // Send request to Python service
            ResponseEntity<AssignmentResponse> response = restTemplate.postForEntity(
                pythonApiUrl,
                request,
                AssignmentResponse.class
            );
            
            // Save to database and return response
            return ResponseEntity.ok(response.getBody());
            
        } catch (Exception e) {
            return ResponseEntity.status(500)
                .body(new ErrorResponse("Error processing assignment", e.getMessage()));
        }
    }
}
```

## Input JSON Format (from Spring Boot)
```json
{
    "auditors": [
        {
            "auditor_id": "A001",
            "latitude": 40.7128,
            "longitude": -74.0060,
            "availability_status": "Available"
        }
    ],
    "stores": [
        {
            "store_id": "S001",
            "latitude": 40.7120,
            "longitude": -74.0052,
            "store_status": "Open",
            "assigned_auditor_id": null
        }
    ]
}
```

## Output JSON Format (to Spring Boot)
```json
{
    "auditors": [
        {
            "auditor_id": "A001",
            "latitude": 40.7128,
            "longitude": -74.0060,
            "availability_status": "Available"
        }
    ],
    "stores": [
        {
            "store_id": "S001",
            "latitude": 40.7120,
            "longitude": -74.0052,
            "store_status": "Open",
            "assigned_auditor_id": "A001"
        }
    ],
    "disruptions": [
        {
            "disruption_id": "D001",
            "store_id": "S001",
            "event_type": "Store Assignment",
            "triggered_on": "2025-11-08 10:30:00",
            "reported_by": "system",
            "reassignment_status": "Implemented"
        }
    ]
}
```

# Spring Boot Model Classes

Here are the Spring Boot model classes needed for the integration:

```java
// Request Models
@Data
@NoArgsConstructor
@AllArgsConstructor
public class AssignmentRequest {
    private List<Auditor> auditors;
    private List<Store> stores;
}

@Data
@NoArgsConstructor
@AllArgsConstructor
public class Auditor {
    private String auditorId;
    private double latitude;
    private double longitude;
    private AuditorStatus availabilityStatus;
    
    public enum AuditorStatus {
        Available,
        Unavailable
    }
}

@Data
@NoArgsConstructor
@AllArgsConstructor
public class Store {
    private String storeId;
    private double latitude;
    private double longitude;
    private StoreStatus storeStatus;
    private String assignedAuditorId;
    
    public enum StoreStatus {
        Open,
        Closed
    }
}

// Response Models
@Data
@NoArgsConstructor
@AllArgsConstructor
public class AssignmentResponse {
    private String status;
    private String code;
    private AssignmentData data;
}

@Data
@NoArgsConstructor
@AllArgsConstructor
public class AssignmentData {
    private List<Auditor> auditors;
    private List<Store> stores;
    private List<Disruption> disruptions;
}

@Data
@NoArgsConstructor
@AllArgsConstructor
public class Disruption {
    private String disruptionId;
    private String storeId;
    private DisruptionType eventType;
    private LocalDateTime triggeredOn;
    private String reportedBy;
    private DisruptionStatus reassignmentStatus;
    
    public enum DisruptionType {
        AUDITOR_UNAVAILABLE,
        STORE_CLOSED,
        STORE_ASSIGNMENT
    }
    
    public enum DisruptionStatus {
        PENDING,
        IMPLEMENTED,
        FAILED
    }
}

// Service Layer
@Service
@Slf4j
public class AuditorAssignmentService {
    private final RestTemplate restTemplate;
    private final AuditorAssignmentRepository repository;
    
    @Value("${python.api.url}")
    private String pythonApiUrl;
    
    @Autowired
    public AuditorAssignmentService(RestTemplate restTemplate, 
                                  AuditorAssignmentRepository repository) {
        this.restTemplate = restTemplate;
        this.repository = repository;
    }
    
    public AssignmentResponse processAssignments(AssignmentRequest request) {
        try {
            // Call Python API
            ResponseEntity<AssignmentResponse> response = restTemplate.postForEntity(
                pythonApiUrl + "/api/process-assignments",
                request,
                AssignmentResponse.class
            );
            
            if (response.getStatusCode() == HttpStatus.OK && response.getBody() != null) {
                // Save results to database
                saveAssignments(response.getBody().getData());
                return response.getBody();
            } else {
                throw new ServiceException("Invalid response from Python service");
            }
            
        } catch (RestClientException e) {
            log.error("Error calling Python service", e);
            throw new ServiceException("Failed to process assignments", e);
        }
    }
    
    private void saveAssignments(AssignmentData data) {
        // Save auditors
        data.getAuditors().forEach(repository::saveAuditor);
        
        // Save stores
        data.getStores().forEach(repository::saveStore);
        
        // Save disruptions
        data.getDisruptions().forEach(repository::saveDisruption);
    }
}

// Controller
@RestController
@RequestMapping("/api/v1")
@Slf4j
public class AuditorAssignmentController {
    
    private final AuditorAssignmentService service;
    
    @Autowired
    public AuditorAssignmentController(AuditorAssignmentService service) {
        this.service = service;
    }
    
    @PostMapping("/assign-auditors")
    public ResponseEntity<AssignmentResponse> assignAuditors(
            @Valid @RequestBody AssignmentRequest request) {
        try {
            AssignmentResponse response = service.processAssignments(request);
            return ResponseEntity.ok(response);
        } catch (ServiceException e) {
            log.error("Error processing assignment", e);
            return ResponseEntity.status(HttpStatus.INTERNAL_SERVER_ERROR)
                .body(new AssignmentResponse("error", "PROCESSING_ERROR", null));
        }
    }
}
```

## Configuration (application.yml)
```yaml
python:
  api:
    url: http://localhost:5000

spring:
  datasource:
    url: jdbc:mysql://localhost:3306/audit_system
    username: your_username
    password: your_password
  jpa:
    hibernate:
      ddl-auto: update
    show-sql: true
```

In [ ]:
# save as app.py
# Install required packages if not already installed:
# pip install flask openai

import os
import json
import time
from flask import Flask, request, jsonify
try:
    import openai
except ImportError:
    import sys, subprocess
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'openai'])
    import openai

app = Flask(__name__)

# Ensure your OPENAI_API_KEY is set in the environment
# NOTE: do NOT store actual API keys inside source files or notebooks.
# The code should read the environment variable named 'OPENAI_API_KEY'.
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise RuntimeError("Please set the OPENAI_API_KEY environment variable before running this notebook.")
openai.api_key = OPENAI_API_KEY

class ValidationError(Exception):
    pass

def validate_auditor(auditor):
    required_fields = ['auditor_id', 'latitude', 'longitude', 'availability_status']
    missing = [f for f in required_fields if f not in auditor]
    if missing:
        raise ValidationError(f"Auditor missing required fields: {', '.join(missing)}")
    if not isinstance(auditor['auditor_id'], str):
        raise ValidationError("auditor_id must be a string")
    if not isinstance(auditor['latitude'], (int, float)):
        raise ValidationError("latitude must be a number")
    if not isinstance(auditor['longitude'], (int, float)):
        raise ValidationError("longitude must be a number")
    if not isinstance(auditor['availability_status'], str):
        raise ValidationError("availability_status must be a string")
    if not -90 <= auditor['latitude'] <= 90:
        raise ValidationError("latitude must be between -90 and 90")
    if not -180 <= auditor['longitude'] <= 180:
        raise ValidationError("longitude must be between -180 and 180")
    if auditor['availability_status'] not in ['Available', 'Unavailable']:
        raise ValidationError("availability_status must be 'Available' or 'Unavailable'")

def validate_store(store):
    required_fields = ['store_id', 'latitude', 'longitude', 'store_status']
    missing = [f for f in required_fields if f not in store]
    if missing:
        raise ValidationError(f"Store missing required fields: {', '.join(missing)}")
    if not isinstance(store['store_id'], str):
        raise ValidationError("store_id must be a string")
    if not isinstance(store['latitude'], (int, float)):
        raise ValidationError("latitude must be a number")
    if not isinstance(store['longitude'], (int, float)):
        raise ValidationError("longitude must be a number")
    if not isinstance(store['store_status'], str):
        raise ValidationError("store_status must be a string")
    if not -90 <= store['latitude'] <= 90:
        raise ValidationError("latitude must be between -90 and 90")
    if not -180 <= store['longitude'] <= 180:
        raise ValidationError("longitude must be between -180 and 180")
    if store['store_status'] not in ['Open', 'Closed']:
        raise ValidationError("store_status must be 'Open' or 'Closed'")

def build_prompt(data):
    """
    Build system + user messages for Chat API.
    The assistant is asked to reply with strict JSON only (no explanatory text).
    """
    system_msg = (
        "You are an assistant that assigns auditors to stores. "
        "You must reply with JSON only (no extra text) in the exact schema described below."
    )
    # Schema/constraints we enforce
    schema_instruction = (
        "Return a JSON object with keys: 'auditors', 'stores', 'disruptions'.\n"
        " - 'auditors': list of objects with at least 'auditor_id' and optionally an 'assigned_store_id' (or null) and 'notes'.\n"
        " - 'stores': list of objects with at least 'store_id' and optionally 'assigned_auditor_id' (or null) and 'notes'.\n"
        " - 'disruptions': list of disruption objects (each with 'type' and 'details').\n"
        "Example shape:\n"
        "{\n"
        "  \"auditors\": [{\"auditor_id\": \"A1\", \"assigned_store_id\": \"S1\", \"notes\": \"...\"}],\n"
        "  \"stores\": [{\"store_id\": \"S1\", \"assigned_auditor_id\": \"A1\", \"notes\": \"...\"}],\n"
        "  \"disruptions\": []\n"
        "}\n"
        "Constraints:\n"
        " - Use null for unassigned fields.\n"
        " - Do not include any explanatory text outside JSON.\n"
    )

    # Provide input data (auditors, stores, disruptions) as JSON (safe to parse)
    user_payload = {
        "task": "Assign available auditors to open stores minimizing travel and respecting availability. You may leave some stores unassigned if no available auditor.",
        "input": {
            "auditors": data.get("auditors", []),
            "stores": data.get("stores", []),
            "disruptions": data.get("disruptions", [])
        },
        "response_format": "strict_json"
    }

    # Return the messages array for Chat API
    messages = [
        {"role": "system", "content": system_msg},
        {"role": "system", "content": schema_instruction},
        {"role": "user", "content": "Input JSON:\n" + json.dumps(user_payload)}
    ]
    return messages

def call_chat_api(messages, model="gpt-4o-mini", max_retries=2):
    """
    Call OpenAI Chat Completions (using the python SDK). Returns the assistant text.
    Retries on transient errors.
    """
    attempt = 0
    while True:
        try:
            resp = openai.ChatCompletion.create(
                model=model,
                messages=messages,
                temperature=0.0,
                max_tokens=1500
            )
            # Extract assistant content (first choice)
            assistant_text = resp["choices"][0]["message"]["content"]
            return assistant_text
        except openai.error.RateLimitError as e:
            attempt += 1
            if attempt > max_retries:
                raise
            time.sleep(1.5 * attempt)
        except openai.error.ServiceUnavailableError as e:
            attempt += 1
            if attempt > max_retries:
                raise
            time.sleep(2 * attempt)
        except Exception:
            # Bubble up other exceptions to be handled by caller
            raise

@app.route('/api/process-assignments', methods=['POST'])
def process_assignments():
    try:
        data = request.get_json()
        if not data:
            return jsonify({'error': 'No JSON data provided', 'status': 'error', 'code': 'INVALID_REQUEST'}), 400

        # Basic structure presence check
        if 'auditors' not in data or 'stores' not in data:
            return jsonify({
                'error': 'Missing required sections',
                'status': 'error',
                'code': 'MISSING_DATA',
                'required_fields': ['auditors', 'stores']
            }), 400

        # Validate input items
        try:
            for auditor in data['auditors']:
                validate_auditor(auditor)
            for store in data['stores']:
                validate_store(store)
        except ValidationError as ve:
            return jsonify({'error': str(ve), 'status': 'error', 'code': 'VALIDATION_ERROR'}), 400

        # Ensure disruptions key exists
        if 'disruptions' not in data:
            data['disruptions'] = []

        # Build prompt and call ChatGPT API
        messages = build_prompt(data)
        try:
            assistant_text = call_chat_api(messages)
        except Exception as e:
            return jsonify({
                'error': 'Error calling Chat API',
                'status': 'error',
                'code': 'CHAT_API_ERROR',
                'details': str(e)
            }), 502

        # The assistant should reply with strict JSON. Parse it.
        try:
            result = json.loads(assistant_text)
        except json.JSONDecodeError:
            # Try to extract JSON substring if the model included surrounding text
            import re
            m = re.search(r'(\{[\s\S]*\})', assistant_text)
            if m:
                try:
                    result = json.loads(m.group(1))
                except Exception as e:
                    return jsonify({
                        'error': 'Model returned malformed JSON',
                        'status': 'error',
                        'code': 'MALFORMED_JSON',
                        'model_output': assistant_text
                    }), 500
            else:
                return jsonify({
                    'error': 'Model did not return JSON',
                    'status': 'error',
                    'code': 'NO_JSON_RETURNED',
                    'model_output': assistant_text
                }), 500

        # Basic result validation
        if not isinstance(result, dict) or not all(k in result for k in ['auditors', 'stores', 'disruptions']):
            return jsonify({
                'error': 'Invalid processing result schema',
                'status': 'error',
                'code': 'INVALID_RESULT',
                'model_output': result
            }), 500

        # Optionally, you can further validate structure of returned auditors/stores
        # Return the successful response
        return jsonify({'status': 'success', 'code': 'SUCCESS', 'data': result})

    except Exception as e:
        return jsonify({'error': 'Internal server error', 'status': 'error', 'code': 'INTERNAL_ERROR', 'message': str(e)}), 500

if __name__ == '__main__':
    print("Starting Flask server...")
    print("API endpoint available at http://localhost:5000/api/process-assignments")
    app.run(host='0.0.0.0', port=5000)


ModuleNotFoundError: No module named 'openai'

In [1]:
print("hello world")

hello world


In [ ]:
# Flask application with API key authentication
import os
import json
from flask import Flask, request, jsonify
from functools import wraps
from datetime import datetime
from math import sqrt
from dotenv import load_dotenv

# Load environment variables
load_dotenv('apikey.env')
API_KEY = os.getenv('API_KEY')
FLASK_SECRET_KEY = os.getenv('FLASK_SECRET_KEY')

# Create Flask application
app = Flask(__name__)
app.config['SECRET_KEY'] = FLASK_SECRET_KEY

def require_api_key(f):
    @wraps(f)
    def decorated_function(*args, **kwargs):
        api_key = request.headers.get('X-API-Key')
        if not api_key or api_key != API_KEY:
            return jsonify({
                'error': 'Invalid or missing API key',
                'status': 'error',
                'code': 'UNAUTHORIZED'
            }), 401
        return f(*args, **kwargs)
    return decorated_function


def log_and_400(response_body):
    """Log request details (headers + body) and return a 400 response with given body."""
    try:
        app.logger.warning("Bad request - reason: %s", response_body.get('error'))
        app.logger.warning("Request headers: %s", dict(request.headers))
        try:
            raw = request.get_data(as_text=True)
        except Exception:
            raw = '<could not read body>'
        app.logger.warning("Request body: %s", raw)
    except Exception as e:
        app.logger.error("Failed to log request for 400 response: %s", e)
    return jsonify(response_body), 400

class ValidationError(Exception):
    """Custom exception for validation errors"""
    pass

def validate_auditor(auditor):
    """Validate auditor data"""
    required_fields = ['auditor_id', 'latitude', 'longitude', 'availability_status']
    missing_fields = [field for field in required_fields if field not in auditor]
    if missing_fields:
        raise ValidationError(f"Auditor missing required fields: {', '.join(missing_fields)}")
    
    # Validate data types (auditor_id is now integer)
    if not isinstance(auditor['auditor_id'], int):
        raise ValidationError("auditor_id must be an integer")
    if not isinstance(auditor['latitude'], (int, float)):
        raise ValidationError("latitude must be a number")
    if not isinstance(auditor['longitude'], (int, float)):
        raise ValidationError("longitude must be a number")
    if not isinstance(auditor['availability_status'], str):
        raise ValidationError("availability_status must be a string")
    
    # Validate value ranges
    if not -90 <= auditor['latitude'] <= 90:
        raise ValidationError("latitude must be between -90 and 90")
    if not -180 <= auditor['longitude'] <= 180:
        raise ValidationError("longitude must be between -180 and 180")
    if auditor['availability_status'] not in ['Available', 'Unavailable']:
        raise ValidationError("availability_status must be 'Available' or 'Unavailable'")

def validate_store(store):
    """Validate store data"""
    required_fields = ['store_id', 'latitude', 'longitude', 'store_status']
    missing_fields = [field for field in required_fields if field not in store]
    if missing_fields:
        raise ValidationError(f"Store missing required fields: {', '.join(missing_fields)}")
    
    # Validate data types (store_id is now integer)
    if not isinstance(store['store_id'], int):
        raise ValidationError("store_id must be an integer")
    if not isinstance(store['latitude'], (int, float)):
        raise ValidationError("latitude must be a number")
    if not isinstance(store['longitude'], (int, float)):
        raise ValidationError("longitude must be a number")
    if not isinstance(store['store_status'], str):
        raise ValidationError("store_status must be a string")
    
    # Validate value ranges
    if not -90 <= store['latitude'] <= 90:
        raise ValidationError("latitude must be between -90 and 90")
    if not -180 <= store['longitude'] <= 180:
        raise ValidationError("longitude must be between -180 and 180")
    if store['store_status'] not in ['Open', 'Closed']:
        raise ValidationError("store_status must be 'Open' or 'Closed'")

def calculate_distance(lat1, lon1, lat2, lon2):
    """Calculate Euclidean distance between two points"""
    return sqrt((lat1 - lat2) ** 2 + (lon1 - lon2) ** 2)

def find_best_auditor(store, auditors):
    """Find suitable auditor based on distance"""
    available_auditors = [a for a in auditors if a['availability_status'] == 'Available']
    
    if not available_auditors:
        return None

    scores = []
    for aud in available_auditors:
        distance = calculate_distance(
            store['latitude'], store['longitude'],
            aud['latitude'], aud['longitude']
        )
        scores.append((aud['auditor_id'], distance))

    best_auditor_id = sorted(scores, key=lambda x: x[1])[0][0]
    return best_auditor_id

def save_auditplan(result, path=None):
    """Save the final audit plan (predicted assignments) to a JSON file."""
    try:
        if path is None:
            path = os.path.join(os.path.dirname(__file__), 'auditplan.json')
        with open(path, 'w', encoding='utf-8') as f:
            json.dump(result, f, indent=2, ensure_ascii=False)
        app.logger.info(f"Audit plan saved to {path}")
        return True
    except Exception as e:
        app.logger.error(f"Failed to save audit plan: {e}")
        return False

@app.route('/api/process-assignments', methods=['POST'])
@require_api_key
def process_assignments():
    try:
        data = request.get_json()
        if not data:
            return log_and_400({
                'error': 'No JSON data provided',
                'status': 'error',
                'code': 'INVALID_REQUEST'
            })
            
        if 'auditors' not in data or 'stores' not in data:
            return log_and_400({
                'error': 'Missing required sections',
                'status': 'error',
                'code': 'MISSING_DATA',
                'required_fields': ['auditors', 'stores']
            })
            
        try:
            for auditor in data['auditors']:
                validate_auditor(auditor)
            for store in data['stores']:
                validate_store(store)
        except ValidationError as ve:
            return log_and_400({
                'error': str(ve),
                'status': 'error',
                'code': 'VALIDATION_ERROR'
            })
            
        disruptions = []
        stores = data['stores']
        auditors = data['auditors']

        for s in stores:
            if 'assigned_auditor_id' in s:
                s.pop('assigned_auditor_id', None)

        for store in stores:
            if store['store_status'] == 'Open' and not store.get('assigned_auditor_id'):
                best_auditor_id = find_best_auditor(store, auditors)
                if best_auditor_id:
                    store['assigned_auditor_id'] = best_auditor_id
                    disruptions.append({
                        "disruption_id": f"D{len(disruptions) + 1:03d}",
                        "store_id": store['store_id'],
                        "event_type": "STORE_ASSIGNMENT",
                        "triggered_on": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                        "reported_by": "system",
                        "reassignment_status": "IMPLEMENTED"
                    })
        
        result = {
            'auditors': data['auditors'],
            'stores': stores,
            'disruptions': disruptions
        }
        
        saved = save_auditplan(result)
        if not saved:
            return jsonify({
                'error': 'Failed to persist audit plan',
                'status': 'error',
                'code': 'PERSISTENCE_ERROR'
            }), 500

        return jsonify({
            'status': 'success',
            'code': 'SUCCESS',
            'data': result
        })
        
    except Exception as e:
        app.logger.error(f"Error processing request: {str(e)}")
        return jsonify({
            'error': 'Internal server error',
            'status': 'error',
            'code': 'INTERNAL_ERROR',
            'message': str(e)
        }), 500

@app.route('/api/health', methods=['GET'])
def health_check():
    return jsonify({
        'status': 'success',
        'message': 'Service is running',
        'timestamp': datetime.now().isoformat()
    })

if __name__ == '__main__':
    if not API_KEY:
        raise ValueError("API_KEY must be set in apikey.env file")
        
    print("Starting Flask server...")
    print("API endpoint will be available at: http://localhost:5000/api/process-assignments")
    app.run(host='0.0.0.0', port=5000)


In [ ]:
#!/usr/bin/env python3
"""
Flask service: receive auditors + stores JSON from Spring Boot, predict one-to-one assignments
(one auditor -> at most one store) and return JSON result.

Environment:
 - Create an `apikey.env` file with keys API_KEY and FLASK_SECRET_KEY (or set them in env)
"""
import os
import json
import time
from typing import List, Dict, Any, Optional
from math import radians, sin, cos, sqrt, asin
from datetime import datetime
from functools import wraps

from dotenv import load_dotenv
from flask import Flask, request, jsonify

# Load environment variables
load_dotenv("apikey.env")
API_KEY = os.getenv("API_KEY")
FLASK_SECRET_KEY = os.getenv("FLASK_SECRET_KEY", "dev-secret")

# Configurable parameters
ESTIMATED_HOURS_PER_STORE = float(os.getenv("ESTIMATED_HOURS_PER_STORE", "4.0"))
DEFAULT_CAPACITY_FOR_AVAILABLE = float(os.getenv("DEFAULT_CAPACITY_FOR_AVAILABLE", "40.0"))
AUTO_ID_START = {"auditor": 1000, "store": 2000}
AUDITOR_STATUS_MAP = {
    "AVAILABLE": "Available",
    "ON_LEAVE": "Unavailable",
    "UNAVAILABLE": "Unavailable",
    "AVAILABLE_PART_TIME": "Available"
}
STORE_STATUS_MAP = {
    "OPEN": "Open",
    "CLOSED": "Closed",
    "OWNERSHIP_CHANGE": "Closed",
    "UNDER_MAINTENANCE": "Closed"
}

# Flask app
app = Flask(__name__)
app.config["SECRET_KEY"] = FLASK_SECRET_KEY

# ---- Utilities ----
def require_api_key(f):
    @wraps(f)
    def decorated_function(*args, **kwargs):
        api_key = request.headers.get("X-API-Key")
        if not API_KEY:
            app.logger.warning("No API_KEY configured on server; rejecting request")
            return jsonify({"error": "Server misconfiguration: API_KEY not set"}), 500
        if not api_key or api_key != API_KEY:
            return jsonify({
                "error": "Invalid or missing API key",
                "status": "error",
                "code": "UNAUTHORIZED"
            }), 401
        return f(*args, **kwargs)
    return decorated_function

class ValidationError(Exception):
    pass

_id_counters = {"auditor": AUTO_ID_START["auditor"], "store": AUTO_ID_START["store"]}
def _next_id(kind: str) -> int:
    _id_counters[kind] += 1
    return _id_counters[kind]

def haversine_km(lat1: float, lon1: float, lat2: float, lon2: float) -> float:
    """Return distance between two lat/lon in kilometers (Haversine)."""
    R = 6371.0  # Earth radius in km
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    return R * c

def save_auditplan(result: dict, path: Optional[str] = None) -> bool:
    try:
        if path is None:
            path = os.path.join(os.path.dirname(__file__), "auditplan.json")
        with open(path, "w", encoding="utf-8") as f:
            json.dump(result, f, indent=2, ensure_ascii=False)
        app.logger.info(f"Audit plan saved to {path}")
        return True
    except Exception as e:
        app.logger.error(f"Failed to save audit plan: {e}")
        return False

# ---- Adaptor: normalize incoming payloads to expected schema ----
def adapt_incoming_payload(raw: Dict[str, Any]) -> Dict[str, List[Dict[str, Any]]]:
    """
    Normalize incoming payload to internal schema:
      auditors: { auditor_id(int), latitude(float), longitude(float),
                  availability_status(str), workloadCapacityHours(float),
                  currentAssignedHours(float), raw: original }
      stores:   { store_id(int), latitude(float), longitude(float),
                  store_status(str), raw: original }
    """
    auditors_in = raw.get("auditors") or raw.get("auditorList") or raw.get("employees") or []
    stores_in = raw.get("stores") or raw.get("storeList") or raw.get("locations") or []

    adapted_auditors = []
    for a in auditors_in:
        auditor_id = a.get("auditor_id") or a.get("id") or a.get("auditorId")
        if auditor_id is None:
            auditor_id = _next_id("auditor")
        try:
            auditor_id = int(auditor_id)
        except Exception:
            auditor_id = _next_id("auditor")

        lat = a.get("latitude") if a.get("latitude") is not None else a.get("homeLat") or a.get("locationLat") or a.get("lat")
        lon = a.get("longitude") if a.get("longitude") is not None else a.get("homeLon") or a.get("locationLon") or a.get("lon")
        try:
            lat = float(lat) if lat is not None else 0.0
            lon = float(lon) if lon is not None else 0.0
        except Exception:
            raise ValidationError("Invalid latitude/longitude for an auditor")

        raw_status = (a.get("availability_status") or a.get("availabilityStatus") or a.get("status") or "").strip()
        availability_status = AUDITOR_STATUS_MAP.get(raw_status.upper(), raw_status or "Unavailable")

        capacity = a.get("workloadCapacityHours") if a.get("workloadCapacityHours") is not None else a.get("capacityHours") or a.get("capacity")
        assigned = a.get("currentAssignedHours") if a.get("currentAssignedHours") is not None else a.get("assignedHours") or a.get("currentAssigned")
        try:
            if capacity is None:
                capacity_val = float(DEFAULT_CAPACITY_FOR_AVAILABLE) if availability_status == "Available" else 0.0
            else:
                capacity_val = float(capacity)
        except Exception:
            capacity_val = 0.0
        try:
            assigned_val = float(assigned) if assigned is not None else 0.0
        except Exception:
            assigned_val = 0.0

        adapted_auditors.append({
            "auditor_id": auditor_id,
            "latitude": lat,
            "longitude": lon,
            "availability_status": availability_status,
            "workloadCapacityHours": capacity_val,
            "currentAssignedHours": assigned_val,
            "raw": a
        })

    adapted_stores = []
    for s in stores_in:
        store_id = s.get("store_id") or s.get("id") or s.get("storeId")
        if store_id is None:
            store_id = _next_id("store")
        try:
            store_id = int(store_id)
        except Exception:
            store_id = _next_id("store")

        lat = s.get("latitude") if s.get("latitude") is not None else s.get("locationLat") or s.get("homeLat") or s.get("lat")
        lon = s.get("longitude") if s.get("longitude") is not None else s.get("locationLon") or s.get("homeLon") or s.get("lon")
        try:
            lat = float(lat) if lat is not None else 0.0
            lon = float(lon) if lon is not None else 0.0
        except Exception:
            raise ValidationError("Invalid latitude/longitude for a store")

        raw_status = (s.get("store_status") or s.get("storeStatus") or s.get("status") or "").strip()
        store_status = STORE_STATUS_MAP.get(raw_status.upper(), raw_status or "Closed")

        adapted_stores.append({
            "store_id": store_id,
            "latitude": lat,
            "longitude": lon,
            "store_status": store_status,
            "raw": s
        })

    return {"auditors": adapted_auditors, "stores": adapted_stores}

# ---- Validators ----
def validate_auditor(a: Dict[str, Any]) -> None:
    required = ["auditor_id", "latitude", "longitude", "availability_status", "workloadCapacityHours", "currentAssignedHours"]
    for r in required:
        if r not in a:
            raise ValidationError(f"Auditor missing required field: {r}")
    if not isinstance(a["auditor_id"], int):
        raise ValidationError("auditor_id must be an integer")
    if not isinstance(a["latitude"], (int, float)) or not -90 <= a["latitude"] <= 90:
        raise ValidationError("Invalid auditor latitude")
    if not isinstance(a["longitude"], (int, float)) or not -180 <= a["longitude"] <= 180:
        raise ValidationError("Invalid auditor longitude")
    if not isinstance(a["availability_status"], str):
        raise ValidationError("availability_status must be a string")
    if not isinstance(a["workloadCapacityHours"], (int, float)):
        raise ValidationError("workloadCapacityHours must be numeric")
    if not isinstance(a["currentAssignedHours"], (int, float)):
        raise ValidationError("currentAssignedHours must be numeric")

def validate_store(s: Dict[str, Any]) -> None:
    required = ["store_id", "latitude", "longitude", "store_status"]
    for r in required:
        if r not in s:
            raise ValidationError(f"Store missing required field: {r}")
    if not isinstance(s["store_id"], int):
        raise ValidationError("store_id must be an integer")
    if not isinstance(s["latitude"], (int, float)) or not -90 <= s["latitude"] <= 90:
        raise ValidationError("Invalid store latitude")
    if not isinstance(s["longitude"], (int, float)) or not -180 <= s["longitude"] <= 180:
        raise ValidationError("Invalid store longitude")
    if not isinstance(s["store_status"], str):
        raise ValidationError("store_status must be a string")

# ---- Core assignment logic (one-to-one) ----
def assign_stores_to_auditors(auditors: List[Dict[str, Any]], stores: List[Dict[str, Any]]) -> Dict[str, Any]:
    """
    Assign OPEN stores to AVAILABLE auditors minimizing travel, with the constraint:
      - each auditor can be assigned to at most one store (one-to-one).
    Auditors with remaining_hours <= 0 are not eligible.
    """
    auditors_map: Dict[int, Dict[str, Any]] = {a["auditor_id"]: dict(a) for a in auditors}
    stores_map: Dict[int, Dict[str, Any]] = {s["store_id"]: dict(s) for s in stores}

    # Prepare auditors: remaining hours and assigned_store_ids (max one)
    for a in auditors_map.values():
        remaining = float(a.get("workloadCapacityHours", 0.0)) - float(a.get("currentAssignedHours", 0.0))
        a["remaining_hours"] = max(0.0, remaining)
        a["assigned_store_ids"] = []

    disruptions = []
    open_stores = [s for s in stores_map.values() if s["store_status"] == "Open"]

    # We'll greedily assign each store to the nearest eligible auditor,
    # and once an auditor is assigned, they are removed from eligibility.
    for store in open_stores:
        eligible = [
            a for a in auditors_map.values()
            if a["availability_status"] == "Available" and a["remaining_hours"] > 0.0 and len(a["assigned_store_ids"]) == 0
        ]
        if not eligible:
            store["assigned_auditor_id"] = None
            disruptions.append({
                "disruption_id": f"D{len(disruptions)+1:03d}",
                "store_id": store["store_id"],
                "event_type": "NO_AVAILABLE_AUDITOR",
                "triggered_on": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                "reported_by": "system",
                "reassignment_status": "PENDING"
            })
            continue

        # compute nearest eligible auditor
        distances = []
        for a in eligible:
            dist_km = haversine_km(store["latitude"], store["longitude"], a["latitude"], a["longitude"])
            distances.append((a["auditor_id"], dist_km))
        distances.sort(key=lambda x: x[1])
        chosen_id, chosen_dist = distances[0]
        chosen = auditors_map[chosen_id]

        # allocate hours (use up to ESTIMATED_HOURS_PER_STORE or whatever remaining)
        alloc_hours = min(ESTIMATED_HOURS_PER_STORE, chosen["remaining_hours"])
        chosen["remaining_hours"] = max(0.0, chosen["remaining_hours"] - alloc_hours)
        chosen["assigned_store_ids"].append(store["store_id"])

        # enforce one-to-one by not allowing further assignments to this auditor:
        # already handled because len(assigned_store_ids) == 1 will block future eligibility

        store["assigned_auditor_id"] = chosen_id

        disruptions.append({
            "disruption_id": f"D{len(disruptions)+1:03d}",
            "store_id": store["store_id"],
            "event_type": "STORE_ASSIGNMENT",
            "triggered_on": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "reported_by": "system",
            "reassignment_status": "IMPLEMENTED",
            "assigned_auditor_id": chosen_id,
            "distance_km": round(chosen_dist, 4),
            "allocated_hours": round(alloc_hours, 3)
        })

    # Build auditors output
    output_auditors = []
    for a in auditors_map.values():
        used_hours = float(a.get("workloadCapacityHours", 0.0)) - a["remaining_hours"]
        predicted_assigned = float(a.get("currentAssignedHours", 0.0)) + used_hours
        output_auditors.append({
            "auditor_id": a["auditor_id"],
            "latitude": a["latitude"],
            "longitude": a["longitude"],
            "availability_status": a["availability_status"],
            "workloadCapacityHours": a["workloadCapacityHours"],
            "currentAssignedHours": round(predicted_assigned, 3),
            "remaining_hours": round(a["remaining_hours"], 3),
            "assigned_store_ids": a["assigned_store_ids"],
            "raw": a.get("raw")
        })

    # Build stores output
    output_stores = []
    for s in stores_map.values():
        output_stores.append({
            "store_id": s["store_id"],
            "latitude": s["latitude"],
            "longitude": s["longitude"],
            "store_status": s["store_status"],
            "assigned_auditor_id": s.get("assigned_auditor_id"),
            "raw": s.get("raw")
        })

    return {"auditors": output_auditors, "stores": output_stores, "disruptions": disruptions}

# ---- API route ----
@app.route("/api/process-assignments", methods=["POST"])
@require_api_key
def process_assignments():
    try:
        data = request.get_json()
        if not data:
            return jsonify({"error": "No JSON data provided", "status": "error", "code": "INVALID_REQUEST"}), 400

        try:
            adapted = adapt_incoming_payload(data)
        except ValidationError as ve:
            return jsonify({"error": str(ve), "status": "error", "code": "ADAPTATION_ERROR"}), 400
        except Exception as e:
            app.logger.exception("Unexpected error during adaptation")
            return jsonify({"error": "Adaptation failed", "status": "error", "code": "ADAPTATION_ERROR", "details": str(e)}), 500

        auditors = adapted["auditors"]
        stores = adapted["stores"]

        try:
            for a in auditors:
                validate_auditor(a)
            for s in stores:
                validate_store(s)
        except ValidationError as ve:
            return jsonify({"error": str(ve), "status": "error", "code": "VALIDATION_ERROR"}), 400

        try:
            result = assign_stores_to_auditors(auditors, stores)
        except Exception as e:
            app.logger.exception("Assignment error")
            return jsonify({"error": "Assignment failed", "status": "error", "code": "ASSIGNMENT_ERROR", "details": str(e)}), 500

        saved = save_auditplan(result)
        if not saved:
            app.logger.warning("Failed to save auditplan to file; continuing response")

        app.logger.info("Assignment result: %s", json.dumps(result, default=str))

        return jsonify({"status": "success", "code": "SUCCESS", "data": result}), 200

    except Exception as e:
        app.logger.exception("Unhandled exception in process_assignments")
        return jsonify({"error": "Internal server error", "status": "error", "code": "INTERNAL_ERROR", "message": str(e)}), 500

@app.route("/api/health", methods=["GET"])
def health_check():
    return jsonify({"status": "success", "message": "Service is running", "timestamp": datetime.now().isoformat()}), 200

if __name__ == "__main__":
    if not API_KEY:
        raise RuntimeError("API_KEY must be set in apikey.env or environment variables")
    print("Starting Flask server...")
    print("API endpoint available at http://0.0.0.0:5000/api/process-assignments")
    app.run(host="0.0.0.0", port=5000)
